# App Profile Recommendation

In this project we analyze the data available from the various apps available on Google Play and the App Store to find out user engagement. As all of the apps are free, the main revenue stream is trhough in-app ads. The more the user interacts with theem, the more profitable the apps become.

The main goal of the project is to find out those apps that are generating most revenue and/or show the most potential for generating higher revenue. In other words, the goal is to help the developers understand what type of apps are likely to attract more users on Google Play and the App Store

# Opening and Exploring the datasets

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, we should first try to see if we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our goals:

A [data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).

A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

As we have the data available to us, we can start by opening the dataset and then explore it.

In [1]:
from csv import reader
opened_file_google = open('dataset\googleplaystore.csv', encoding="utf8")
read_file_google = reader(opened_file_google)
apps_data_google = list(read_file_google)
google_header = apps_data_google[0]
google_data = apps_data_google[1:]

opened_file_apple = open('dataset\AppleStore.csv', encoding="utf8")
read_file_apple = reader(opened_file_apple)
apps_data_apple = list(read_file_apple)
apple_header = apps_data_apple[0]
apple_data = apps_data_apple[1:]

To make the dataset explore easily, a function `explore()` is created below. It takes as input the dataset to be explored along with the start row and end row for the exploration. It can also provide us with the information such as the total number of rows and columns the dataset has. This function can be used to explore the data repeatedly with great readability.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

We are ready to use the `explore()` function to start exploring the data. Below we printed out first 5 rows for each dataset. First, the Google store data and then the Apple play store data. 

As can be seen from below, Google play store dataset has total **10842** rows including the header row (that has all the column names) and **13** columns in total

In [3]:
explore_data(google_data, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


On the other hand, Apple store dataset has **7198** rows including the header row and a grand total of **16** columns.

In [4]:
explore_data(apple_data, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


Let us now take a look at the column names (header row) for each dataset separately. 

Firstly, we look at the column names of the Google play store dataset. A detailed description of the columns are given in the [documentation](https://www.kaggle.com/lava18/google-play-store-apps).

Some of the useful columns for our analysis can be `'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Content Rating', 'Genres'`.

In [5]:
print(google_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Now let us take a glance at the column names of the Apple store dataset. A detailed description of the columns are given in the [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps). 

From these set of columns, it looks like `'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'prime_genre'` are some of the columns that can be useful for our analysis.

In [6]:
print(apple_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
